#### Calculando Coeficiente Dice

In [21]:
import numpy as np
import glob

from utils import load_images_array

In [22]:
# Carregando as imagens
ORIGINAL_SIZE = 850
NEW_SIZE = 256

_folder = './TM40_Original'

GT_imgs = sorted(glob.glob('GT_Producao/*'))
pred_imgs = sorted(glob.glob('pred_imgs/*'))

In [40]:
# Carregando as imagens em array
gt = load_images_array(GT_imgs, new_size = NEW_SIZE)
pred = load_images_array(pred_imgs, new_size = NEW_SIZE)

#### Note que o formato do input utilizado no laboratório é idêntico ao utilizado por Vitor e Eduardo com o np.stack

In [41]:
# Utilizado no laboratório
print(f'Tipo: {type(pred)} | Formato: {pred.shape}') 

Tipo: <class 'numpy.ndarray'> | Formato: (408, 256, 256, 1)


In [42]:
gt = np.stack(gt)
pred = np.stack(pred)

# Utilizado por Vitor e Eduardo
print(f'Tipo: {type(pred)} | Formato: {pred.shape}') # Note que não há diferença no output

Tipo: <class 'numpy.ndarray'> | Formato: (408, 256, 256, 1)


#### Note que o resultado do cálculo do coeficiente Dice utilizando a versão do Vitor e Eduardo (0.6888) foi igual a utilizada na segmentação das tomografias de pulmão, enquanto a versão utilizada no trabalho do Girino apresentou resultado divergente (0.7284), que também apresentou um tempo de execução mais que 5x maior do que as outras duas versões.

##### Cálculo Dice (Versão Eduardo e Vitor)

In [27]:
from keras import *

In [28]:

def dice_coef(y_true, y_pred):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2.*intersection+1)/(backend.sum(y_true_f)+backend.sum(y_pred_f)+1)

In [50]:
dice = dice_coef(gt, pred)
dice = str(dice).split(',')[0].split('(')[1]
print(f'Coeficiente Dice (Eduardo e Vitor): {dice}')

Coeficiente Dice (Eduardo e Vitor): 0.6888157729909007


##### Cálculo Dice (Girino Version)

In [30]:
from sklearn.metrics import jaccard_score

In [31]:
def jaccard_coef(y_true, y_pred):
    
    # Recebe dois 2d-array de bool
    
    # array_true = asarray(y_true)
    # array_pred = asarray(y_pred)
    iou = jaccard_score(y_true.flatten(),y_pred.flatten())
    return iou


def dice_coef_2(y_true, y_pred):
    
    # Recebe dois 2d-array de bool
    
    iou = jaccard_coef(y_true, y_pred)
    dice = (2*iou)/(iou+1)
    return dice


def calcula_metricas(Y_test, predicao):
    
    # Recebe predicao e Y_test (no padrao keras, do tipo (78, 256, 256, 1) )
    
    # Reshape e transforma em boolean
    Y_test = Y_test[:,:,:,0]
    Y_test_bool = Y_test > 0.5 #
    
    # Reshape e transforma em boolean
    predicao = predicao[:,:,:,0] #
    predicao_bool = predicao > 0.5 #
    
    #iou = np.zeros(predicao.shape[0])
    #dice = np.zeros(predicao.shape[0])
    
    iou_list = []
    dice_list = []
    
    for i in range(predicao.shape[0]):
        iou_list.append(jaccard_coef(Y_test_bool[i], predicao_bool[i]))
        dice_list.append(dice_coef_2(Y_test_bool[i], predicao_bool[i]))
    
    return dice_list

In [51]:
# Calcula iou e dice para todas as imagens deste fold (i)
dice_list = []
dice_list = calcula_metricas(gt, pred)  

#estatística
mean = round(np.mean(dice_list), 4)
print(f'Coeficiente Dice (Versão do Laboratório): {mean}')

Coeficiente Dice (Versão do Laboratório): 0.7284


##### Cálculo Dice (Lung Version)

In [33]:
import keras.backend as K

In [34]:
def dice_coef_lung(y_true, y_pred): #Sørensen–Dice coefficient
    y_true_f = K.flatten(y_true) #transformando em unidimensional
    y_pred_f = K.flatten(y_pred) #transformando em unidimensional
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + \
        K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

In [55]:
dice = dice_coef_lung(gt, pred)
dice = str(dice).split(',')[0].split('(')[1]
print(f'Coeficiente Dice (Versão usada com as tomografias de pulmão): {dice}')

Coeficiente Dice (Versão usada com as tomografias de pulmão): 0.6888156171481549
